## nfl lines machine learning project


source data for historical spreads and game scores
http://www.repole.com/sun4cast/data.html

In [1]:
import os

# define the root directory for the nfl code in $MLNLF_ROOT

#codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
codeDir = "".join(['/Users/amit/repos/mlnfl/nfl', os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

/Users/amit/repos/mlnfl/nfl/
/Users/amit/repos/mlnfl/nfl/data/


In [2]:
# warnings control
import warnings
# choose default, ignore, always
warnings.filterwarnings('default')


In [3]:
# import necessary modules
%matplotlib inline

from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import madden

from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble

print ("pandas version ",pd.__version__)

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationW

pandas version  0.18.0


In [4]:
# location of lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit/repos/mlnfl/nfl/data/lookup/


In [5]:
# import reference data
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,Ravens,afc,north,NaN
New England Patriots,NaN,Patriots,afc,east,NaN
Tennessee Titans,NaN,Titans,afc,south,NaN
Atlanta Falcons,NaN,Falcons,nfc,south,NaN
Tampa Bay Buccaneers,NaN,Buccaneers,nfc,south,NaN


In [6]:
###  multi-season training
reload(madden)

# testYears and trainYears need to be arrays
trainYears = [2014] #range(2008,2013) 
testYear = [2015]

# train on previous 3 yrs of data
testYear = [2015]
trainYears = range(testYear[0]-3,testYear[0]) 

# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

training seasons >>  [2012 2013 2014]


In [7]:
reload(madden)

# get training data
# 1 - read all the games
path_to_lines = dataRoot + "lines/"
dfAllGames = madden.readGamesAll(path_to_lines, seasons)
# 2 - compile season record for all teams
dfAllTeams = madden.seasonRecord(dfAllGames, reference_data)
# 3 - apply season records and compute other fields for all games
dfAllGames = madden.processGames(dfAllGames, dfAllTeams, reference_data)
# 4 - remove extra year of data 
dfAllGames = dfAllGames[dfAllGames.season.isin(seasons)]

# use different test set 
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)
# 1 - read all the games
dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)  
# 2 - compile season record for all teams
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data) 
# 3 - apply season records and compute other fields for all games
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data) 
# 4 - remove extra year of data 
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]


madden.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['gamesPlayed'] = range(1, len(team_df.index) + 1)  # index 1 thur 16
madden.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['team'] = team
madden.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['homeGame'] = season_df['Hom

results for >>  [2015]


In [34]:
# define independent variables for logistic regression
features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]

# run the classifer
random_state = 11
#classifier = svm.SVC(kernel='poly',probability=True, random_state=random_state)
classifier = linear_model.LogisticRegression(C=1e5)

mlClassifier = madden.runScikitClassifier(dfAllGames,features,classifier)

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfGamesTest,mlClassifier,features)

# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])

In [14]:
dfAllGames

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,week,favorite,...,homeWin,favoredWin,gameWeek,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine
7918,2012-09-05,Dallas Cowboys,24.0,New York Giants,17.0,4.0,46,2012,NaN,New York Giants,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0.5625,0.5000,4.0
7919,2012-09-09,Indianapolis Colts,21.0,Chicago Bears,41.0,10.0,42.5,2012,NaN,Chicago Bears,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.1250,10.0
7920,2012-09-09,Philadelphia Eagles,17.0,Cleveland Browns,16.0,-9.5,42,2012,NaN,Philadelphia Eagles,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.2500,9.5
7921,2012-09-09,Buffalo Bills,28.0,New York Jets,48.0,3.0,39,2012,NaN,New York Jets,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.3750,3.0
7922,2012-09-09,Washington Redskins,40.0,New Orleans Saints,32.0,9.0,50.5,2012,NaN,New Orleans Saints,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0.8125,0.3125,9.0
7923,2012-09-09,New England Patriots,34.0,Tennessee Titans,13.0,-6.5,48,2012,NaN,New England Patriots,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.8125,0.5625,6.5
7924,2012-09-09,Jacksonville Jaguars,23.0,Minnesota Vikings,26.0,3.5,38,2012,NaN,Minnesota Vikings,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.1875,0.3125,3.5
7925,2012-09-09,Miami Dolphins,10.0,Houston Texans,30.0,13.5,41.5,2012,NaN,Houston Texans,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.3750,13.5
7926,2012-09-09,Los Angeles Rams,23.0,Detroit Lions,27.0,9.0,46.5,2012,NaN,Detroit Lions,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.1250,9.0
7927,2012-09-09,Atlanta Falcons,40.0,Kansas City Chiefs,24.0,-2.5,43,2012,NaN,Atlanta Falcons,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.4375,2.5


In [22]:
features

['favoredRecord',
 'underdogRecord',
 'prevFavoredRecord',
 'prevUnderdogRecord',
 'gameWeek',
 'absLine',
 'divisionGame',
 'favoredHomeGame']

In [24]:
dfAllGames.to_csv(dataRoot+ 'nfldata_2012_2014.csv', index=False)
dfGamesTest.to_csv(dataRoot+ 'nfldata_2015.csv', index=False)

In [9]:
# diagnostics ... weekly outcomes
g = dfAll.groupby('gameWeek')['lineScore','probaScore1','probaScore2','probaScore3'].sum()
print(g)

g.sum()

          lineScore  probaScore1  probaScore2  probaScore3
gameWeek                                                  
1             104.0        100.0        103.0        100.0
2              45.0         62.0         68.0         70.0
3             110.0        102.0         94.0         95.0
4             105.0        100.0         95.0         94.0
5              98.0        104.0         97.0         97.0
6              92.0         91.0         94.0         93.0
7             100.0        111.0        111.0        111.0
8             105.0        106.0        113.0        115.0
9              78.0         83.0         82.0         84.0
10             34.0         41.0         48.0         51.0
11             83.0         84.0         97.0        101.0
12             88.0         95.0         91.0         94.0
13             95.0        100.0        101.0         99.0
14             92.0         96.0        103.0         97.0
15            117.0        119.0        119.0        121

lineScore      1504.0
probaScore1    1553.0
probaScore2    1576.0
probaScore3    1583.0
dtype: float64

In [30]:
# predict one week of current season
iweek = 2
reload(madden)

# use different test set - current year
testYear = [2015]
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)

dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# pick only this weeks games for predict
dfTest = dfGamesTest[dfGamesTest.gameWeek == iweek]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfTest,mlClassifier,features)
# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict, reference_data, seasonTest[0])



results for >>  [2015]


In [43]:
# display weekly ranking output

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

dispCols = ['season','gameWeek','Visitor','visitorRecord','Home Team','homeRecord',
            'Line','prevFavoredRecord','prevUnderdogRecord','predict_proba',
            'lineGuess','probaGuess', 'probaAbsGuess', 'predictTeam']


dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
guessCol = 'probaGuess'
predictCols = ['gameWeek','predictTeam', 'predict_proba', guessCol, 'favorite','lineGuess', 'Line']
dfAll[predictCols].sort(guessCol, ascending=False)

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8928,17,Carolina Panthers,0.900338,16.0,Carolina Panthers,16.0,10.5
8686,1,New England Patriots,0.705755,16.0,New England Patriots,16.0,7.0
8756,5,Green Bay Packers,0.862176,16.0,Green Bay Packers,15.0,9.0
8745,4,Arizona Cardinals,0.866069,16.0,Arizona Cardinals,12.0,7.0
8773,6,Green Bay Packers,0.906347,16.0,Green Bay Packers,16.0,10.5
8786,7,New England Patriots,0.864424,16.0,New England Patriots,16.0,9.0
8723,3,New England Patriots,0.925331,16.0,New England Patriots,15.0,13.5
8791,8,New England Patriots,0.882147,16.0,New England Patriots,15.0,8.0
8807,9,New England Patriots,0.939148,16.0,New England Patriots,16.0,14.0
8819,10,Green Bay Packers,0.922728,16.0,Green Bay Packers,16.0,11.5


In [45]:
dfAll[predictCols].to_csv(dataRoot + 'predict2015.csv', index=False)

In [42]:
week_filter = dfAll.gameWeek >=1

dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
dfAll[week_filter][predictCols].sort(guessCol, ascending=False)

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


KeyError: 'probaGuess'

In [40]:
# display weekly ranking output for spread method

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

predictCols = ['favorite','lineGuess', 'absLine','Line', 'favoredHomeGame', 'divisionGame', 'favoredRecord', 'gameWeek']

sortCols = ['gameWeek', 'absLine','favoredHomeGame', 'divisionGame', 'favoredRecord', 'favorite']
dfSpread = dfAll[predictCols].sort(sortCols , ascending=True)
#print(dfSpread.to_csv(sys.stdout,sep=',', index=False))
dfSpread

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,favorite,lineGuess,absLine,Line,favoredHomeGame,divisionGame,favoredRecord,gameWeek
8688,Houston Texans,1.0,1.0,1.0,1,0,0.000000,1
8690,Indianapolis Colts,2.0,2.5,-2.5,0,0,0.000000,1
8701,Minnesota Vikings,3.0,2.5,-2.5,0,0,0.000000,1
8694,Arizona Cardinals,4.0,2.5,2.5,1,0,0.000000,1
8692,Carolina Panthers,6.0,3.0,-3.0,0,0,0.000000,1
8700,Philadelphia Eagles,5.0,3.0,-3.0,0,0,0.000000,1
8689,New York Jets,7.0,3.0,3.0,1,0,0.000000,1
8695,San Diego Chargers,8.0,3.0,3.0,1,0,0.000000,1
8696,Tampa Bay Buccaneers,9.0,3.0,3.0,1,0,0.000000,1
8697,Cincinnati Bengals,10.0,3.5,-3.5,0,0,0.000000,1


In [41]:
dfSpread.to_csv(dataRoot + 'predict2015.csv', index=False)

In [30]:
# loop over multiple years

import runMadden
import walsh
reload(madden)
reload(runMadden)


trainStart = 2010
trainLen = 3

random_state = None
mlClassifier = svm.SVC(kernel='poly',probability=True, random_state=random_state)
dfSVM = runMadden.runSeasonLoop(trainStart, trainLen, mlClassifier, path_to_lines, reference_data)

mlClassifier = linear_model.LogisticRegression(C=1e5)
dfLog = runMadden.runSeasonLoop(trainStart, trainLen, mlClassifier, path_to_lines, reference_data)
#dfLoop = dfSVM


2013 [2010, 2011, 2012]
1653 <type 'int'>
2014 [2011, 2012, 2013]
1664 <type 'int'>
2013 [2010, 2011, 2012]
1653 <type 'int'>
2014 [2011, 2012, 2013]
1664 <type 'int'>


In [31]:
dfSVM

,lineScore,probaScore1,probaScore2,probaScore3,trainYears,classifierType,classifier
season,,,,,,,
2013,2,8,2,-17,[2010 2011 2012],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."
2014,5,85,99,113,[2011 2012 2013],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."


In [32]:
dfLog

,lineScore,probaScore1,probaScore2,probaScore3,trainYears,classifierType,classifier
season,,,,,,,
2013,2,28,40,35,[2010 2011 2012],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
2014,5,5,-35,-47,[2011 2012 2013],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
